after made some labels using lenslet, we can extract the labels and use them to train a model:

In [1]:
import unibox as ub

rated_dirs = [
    "/home/ubuntu/dev/lenslet/lenscat-backend/data/distill_dit_hires_user-prompt_bestof16-m1-aes12/images/batch_0",
    "/home/ubuntu/dev/lenslet/lenscat-backend/data/distill_dit_hires_user-prompt_bestof16-m1-aes12/images/batch_1",
]

# Collect all .png.json metadata files
all_metas = [m for d in rated_dirs for m in ub.ls(d, [".png.json"], debug_print=False)]

# Load all metadata concurrently
meta_contents = ub.concurrent_loads(all_metas)
path_meta_dict = dict(zip(all_metas, meta_contents))

# Filter only those with star == 5
filtered = {p: m for p, m in path_meta_dict.items() if m.get("star") == 5}
ub.peeks(filtered)

# Build list of image and JSON paths to upload
json_paths = list(filtered.keys())
img_paths = [p.removesuffix(".json") for p in json_paths]  # works for .png.json
all_paths = img_paths + json_paths
ub.peeks(all_paths)

Listing local files: 0files [00:00, ?files/s]

Listing local files: 0files [00:00, ?files/s]

Loading concurrent:   0%|          | 0/4000 [00:00<?, ?it/s]

{'type': 'list',
 'length': 210,
 'head': ['/home/ubuntu/dev/lenslet/lenscat-backend/data/distill_dit_hires_user-prompt_bestof16-m1-aes12/images/batch_0/948.png',
  '/home/ubuntu/dev/lenslet/lenscat-backend/data/distill_dit_hires_user-prompt_bestof16-m1-aes12/images/batch_0/743.png',
  '/home/ubuntu/dev/lenslet/lenscat-backend/data/distill_dit_hires_user-prompt_bestof16-m1-aes12/images/batch_0/938.png']}

In [ ]:
cp_commands = [f'aws s3 cp {p} s3://dataset-ingested/temp/lenslet-rated-5stars/' for p in all_paths]

import os
for cmd in cp_commands:
    os.system(cmd)